## A full business solution

## Business Challenge:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will provide a company name and their primary website.

In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problen with your API key? Please troubleshoot")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage
# Common headers (mimic a browser request)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/117.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Connection": "keep-alive",
    "Referer": "https://www.google.com/",
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
    

In [4]:
ed = Website("https://www.labcorp.com/")
ed.links

['#main-content',
 'https://www.labcorp.com/',
 'https://www.labcorp.com/logins',
 'https://www.labcorp.com/labs-and-appointments-advanced-search',
 'https://www.labcorp.com/patients/results',
 'https://patient.labcorp.com/invoices/find',
 'https://www.ondemand.labcorp.com/products',
 'https://www.labcorp.com/patients',
 'https://www.labcorp.com/test-menu/search',
 'https://link.labcorp.com/landing.html',
 'https://www.labcorp.com/science',
 'https://www.labcorp.com/contact-labcorp-account-representative',
 'https://www.labcorp.com/providers',
 'https://www.labcorp.com/organizations/organizations-we-serve/hospitals',
 'https://www.labcorp.com/organizations/capabilities/employee-testing-wellness',
 'https://www.labcorp.com/organizations/organizations-we-serve/health-plans-and-payors',
 'https://www.labcorp.com/organizations/resources',
 'https://www.labcorp.com/organizations',
 'https://www.labcorp.com/biopharma/nonclinical',
 'https://www.labcorp.com/biopharma/central-labs',
 'https://

## First step: Have GPT-4o-mini figure out which links are relevant

# use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.
It should decide which links are relevant and replace relative links such as "/about" with  "https://company.com/about"
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.labcorp.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#main-content
https://www.labcorp.com/
https://www.labcorp.com/logins
https://www.labcorp.com/labs-and-appointments-advanced-search
https://www.labcorp.com/patients/results
https://patient.labcorp.com/invoices/find
https://www.ondemand.labcorp.com/products
https://www.labcorp.com/patients
https://www.labcorp.com/test-menu/search
https://link.labcorp.com/landing.html
https://www.labcorp.com/science
https://www.labcorp.com/contact-labcorp-account-representative
https://www.labcorp.com/providers
https://www.labcorp.com/organizations/organizations-we-serve/hospitals
https://www.labcorp.com/organizations/capabilities/employee-testing-wellness
https://www.labcorp.com/organizations/organizations-we-s

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
get_links("https://www.labcorp.com/")

{'links': [{'type': 'about page', 'url': 'https://www.labcorp.com/about'},
  {'type': 'careers page', 'url': 'https://careers.labcorp.com/global/en'},
  {'type': 'company page', 'url': 'https://www.labcorp.com/organizations'},
  {'type': 'newsroom', 'url': 'https://www.labcorp.com/newsroom'}]}

## Second step : make the brochure
Assemble all the details into another prompt GPT4-o

In [11]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [12]:
print(get_all_details("https://www.labcorp.com/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.labcorp.com/about'}, {'type': 'careers page', 'url': 'https://careers.labcorp.com/global/en'}, {'type': 'newsroom', 'url': 'https://www.labcorp.com/newsroom'}, {'type': 'company page', 'url': 'https://www.labcorp.com/organizations'}, {'type': 'company page', 'url': 'https://www.labcorp.com/science'}, {'type': 'company page', 'url': 'https://www.labcorp.com/biopharma'}]}
Landing page:
Webpage Title:
Lab Diagnostics & Drug Development, Global Life Sciences Leader | Labcorp
Webpage Contents:
Skip to main content
Logins
Individuals & Patients
Find a Lab
View Test Results
Pay a Bill
Shop for Tests
View Individuals & Patients Page
Providers
Test Menu
Provider Login
Education & Experts
Contact Us
View Providers Page
Health Systems & Organizations
Hospitals & Health Systems
Employee Wellness & Testing
Managed Care & Payors
Resources
View Organizations Page
Biopharma & Investigators
Nonclinical Research
Central Laboratory Servic

In [13]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [15]:
get_brochure_user_prompt("Labcorp", "https://www.labcorp.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.labcorp.com/about'}, {'type': 'careers page', 'url': 'https://careers.labcorp.com/global/en'}, {'type': 'newsroom', 'url': 'https://www.labcorp.com/newsroom'}, {'type': 'company page', 'url': 'https://www.labcorp.com/'}]}


"You are looking at a company called: Labcorp\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nLab Diagnostics & Drug Development, Global Life Sciences Leader | Labcorp\nWebpage Contents:\nSkip to main content\nLogins\nIndividuals & Patients\nFind a Lab\nView Test Results\nPay a Bill\nShop for Tests\nView Individuals & Patients Page\nProviders\nTest Menu\nProvider Login\nEducation & Experts\nContact Us\nView Providers Page\nHealth Systems & Organizations\nHospitals & Health Systems\nEmployee Wellness & Testing\nManaged Care & Payors\nResources\nView Organizations Page\nBiopharma & Investigators\nNonclinical Research\nCentral Laboratory Services\nOrder a Kit\nContact Us\nView Biopharma Section\nManaging Your Health\nShop for Health Tests\nExplore Women's Health\nAnnual Wellness Guidelines\nMore\nDiseases & Therapeutic Areas\nCancer & Oncology\nNeurology\nRheumat

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt },
            {"role": "system", "content": get_brochure_user_prompt(company_name, url)}
        ]
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [17]:
create_brochure("Labcorp", "https://www.labcorp.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.labcorp.com/about'}, {'type': 'careers page', 'url': 'https://careers.labcorp.com/global/en'}, {'type': 'company news', 'url': 'https://www.labcorp.com/newsroom'}, {'type': 'organization page', 'url': 'https://www.labcorp.com/organizations'}, {'type': 'science page', 'url': 'https://www.labcorp.com/science'}]}


# Labcorp Brochure

## Welcome to Labcorp
Labcorp is a global leader in life sciences, offering comprehensive diagnostic and drug development services. We are dedicated to advancing healthcare by providing accurate and timely lab results that help patients make informed decisions.

---

## Our Services
### Lab Diagnostics
- **Comprehensive Testing:** Our wide-ranging test menu covers various health areas including cancer, neurology, rheumatology, and infectious diseases.
- **At-Home Testing Kits:** Convenient options for consumers to manage their health from home, including DNA paternity tests and wellness blood tests.

### Drug Development
- **Biopharma Services:** Providing support for nonclinical research, early-phase development, regulatory compliance, and data commercialization.
- **Innovative Solutions:** Partnering with biopharmaceutical companies to advance drug therapies through cutting-edge research.

### Health Solutions for Organizations
- **Employee Wellness Programs:** Offering workplace drug testing, biometric screenings, and health coaching tailored for corporate wellness initiatives.
- **Managed Care:** Facilitating value-based care solutions through data analytics and quality improvement programs.

---

## Company Culture
At Labcorp, we pride ourselves on being a team-oriented and innovative company. Our culture emphasizes collaboration, inclusivity, and a commitment to excellence. We foster a supportive environment where every employee can grow and contribute to our mission of advancing healthcare.

### Diversity and Inclusion
We actively promote diversity within our workforce and believe that diverse teams drive innovation. Our inclusive practices help create an environment where everyone feels valued and empowered to share their unique perspectives.

---

## Careers at Labcorp
### Opportunities
Labcorp is always looking for talented individuals across various fields including:
- Laboratory Science
- Research & Development
- Business Development
- Corporate Operations 

We offer competitive benefits and career advancement opportunities to ensure our employees thrive.

### Join Our Team
If you are passionate about making a difference in healthcare, we encourage you to explore our current career openings and become part of our mission to connect science and healthcare.

---

## Who We Serve
Labcorp serves a broad customer base:
- **Patients and Individuals:** Delivering accessible testing solutions for personal health management.
- **Healthcare Providers:** Offering advanced lab testing services and support for clinical decision-making.
- **Biopharma Companies:** Partnering from research to commercialization to facilitate breakthrough therapies.
- **Corporations and Health Systems:** Providing comprehensive solutions that enhance employee health and operational efficiency.

---

## Contact Us
For more information about our services, to find a laboratory near you, or to explore career opportunities, please visit our website at [www.labcorp.com](https://www.labcorp.com) or contact our customer service.

**Labcorp - Together, advancing health and improving lives.**

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [19]:
stream_brochure("Labcorp", "https://www.labcorp.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.labcorp.com/about'}, {'type': 'careers page', 'url': 'https://careers.labcorp.com/global/en'}, {'type': 'newsroom', 'url': 'https://www.labcorp.com/newsroom'}, {'type': 'company page', 'url': 'https://www.labcorp.com/organizations'}]}



# Welcome to Labcorp

## About Us
Labcorp is a global leader in life sciences, providing comprehensive lab diagnostics and drug development services. We are committed to advancing health for all through innovative diagnostic testing, clinical trials, and drug development solutions. Serving a wide array of industries, including healthcare, pharma, medtech, and agriculture, our mission is to enhance the health and quality of life for patients around the world.

## Our Services
- **Lab Diagnostics:** A vast menu of tests covering diseases and conditions such as cancer, neurology, and autoimmune disorders.
- **Drug Development:** Comprehensive services from early-phase development to regulatory compliance and commercialization.
- **Biopharma Solutions:** Partnering with biopharma companies for clinical research and testing.
- **Employee Wellness Programs:** Custom solutions for workplaces focused on health and safety.

## Innovation & Expertise
Labcorp excels in various scientific disciplines including genetics, toxicology, and digital pathology. Our dedication to precision medicine and cutting-edge technologies enables us to provide accurate testing and effective solutions for managing health. 

## Our Culture
At Labcorp, we prioritize a culture of collaboration, innovation, and respect. Our team is passionate about making a difference in the lives of individuals and communities. We value diverse perspectives and are committed to fostering an inclusive environment where all employees feel empowered to contribute to our mission.

## Careers at Labcorp
Join our team of dedicated professionals! Labcorp offers a range of career opportunities across various disciplines. We provide robust professional development resources, a supportive work environment, and competitive benefits. Whether you’re an experienced scientist, a healthcare professional, or seeking an entry-level position, we encourage you to explore our career opportunities!

### Current Openings
- Clinical Laboratory Technologists
- Research Scientists
- Data Analysts
- Sales Representatives
- Project Managers

## Our Customers
We serve a broad spectrum of clients including individual patients, healthcare providers, health systems, and biopharma companies. Our collaborative approach ensures tailored services that meet the unique needs of each customer segment.

## Get in Touch
For inquiries, support, or to learn more about our services, please visit our website or contact us directly. We’re here to answer your questions and help you navigate your health needs effectively.

### Contact Us: [Labcorp's Contact Page](#)

---

**Labcorp** - Advancing health together. Let’s move science and healthcare forward!
